In [1]:
# !pip install -q mediapipe==0.10.0
!pip install --upgrade -q gspread

from google.colab.patches import cv2_imshow
from google.colab import auth
import gspread
import pandas as pd
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor
from skimage import io, transform
from tqdm import tqdm
auth.authenticate_user()
from google.auth import default
creds, _ = default()
auth = gspread.authorize(creds)



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
print(auth.open('hands and palms loras'))

<Spreadsheet 'hands and palms loras' id:1GHwH5am7o7RKga8FPruSKLgMTXIWG_wEMKk__L0J72A>


In [4]:
worksheet = auth.open('hands and palms loras').worksheet('scale1').get("A2:I625")
df = pd.DataFrame.from_records(worksheet)
df = df.rename({0:'rank', 1:'checkpoint', 2:'side',3:'gender', 4:'hand', 5:'image', 6: 'bad', 7:'ok', 8:'good'}, axis=1)
df



,rank,checkpoint,side,gender,hand,image,bad,ok,good
0,64,30000,front,male,right,5,1,None,None
1,64,30000,front,male,right,4,1,None,None
2,64,30000,front,male,right,3,1,None,None
3,64,30000,front,male,right,2,,1,None
4,64,30000,front,male,right,1,1,None,None
...,...,...,...,...,...,...,...,...,...
619,None,0,back,female,left,4,1,None,None
620,None,0,back,female,left,3,,1,None
621,None,0,back,female,left,2,1,None,None
622,None,0,back,female,left,1,1,None,None


In [5]:
!pip install torchmetrics[image]
!pip install torch-fidelity

In [6]:
# !sudo fallocate -l 64G /tmp/swapfile
# !sudo chmod 600 /tmp/swapfile
# !sudo mkswap /tmp/swapfile
# !sudo swapon /tmp/swapfile


In [7]:
df.loc[df['checkpoint'] == '5000']

,rank,checkpoint,side,gender,hand,image,bad,ok,good
30,64,5000,front,male,right,5,1,None,None
31,64,5000,front,male,right,4,1,None,None
32,64,5000,front,male,right,3,1,None,None
33,64,5000,front,male,right,2,,,1
34,64,5000,front,male,right,1,1,None,None
...,...,...,...,...,...,...,...,...,...
571,4,5000,back,female,left,4,,1,None
572,4,5000,back,female,left,3,,1,None
573,4,5000,back,female,left,2,,,1
574,4,5000,back,female,left,1,1,None,None


In [10]:
import torch_fidelity
from tqdm import tqdm
from PIL import Image
import numpy as np
import torchvision
import torch
import os
from tqdm import tqdm
import random
import gc
from torch_fidelity.fidelity import main
from functools import partial

def calculate_clip_score(image, prompts):
    # images_int = (images * 255).astype("uint8")
    # clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    clip_score = clip_score_fn(torchvision.transforms.functional.pil_to_tensor(image).unsqueeze(0), prompts).detach()
    return float(clip_score)

def getpath(row):
  if row['rank'] != 'None':
    return f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt{row["checkpoint"]}_steps30_{row["image"]}.png'
  else:
    return f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt5000_steps30_{row["image"]}.png'


from torchmetrics.image.kid import KernelInceptionDistance

dataset_path = '/content/drive/MyDrive/cs236 project/hands and palms/Hands/Hands'
real_paths = sorted([os.path.join(dataset_path, x) for x in os.listdir(dataset_path) if not x.endswith('.csv')])
random.seed(1234)
random.shuffle(real_paths)
real_paths = real_paths[:128]
real_images = []
for path in tqdm(real_paths):
  real_images.append(torch.tensor(np.array(Image.open(path).convert("RGB"))).unsqueeze(0).permute(0, 3, 1, 2))
gc.collect()
real_images = torch.cat(real_images)
# real_images.to('cuda')

results = {"checkpoint":[], "r4":[], "r64":[]}
for checkpoint in [0, 5000, 10000, 15000, 20000, 25000, 30000]:
  for rank in [4, 64]:
    if checkpoint ==0 and rank == 64:
      continue
    if checkpoint ==0:
      samples = df.loc[(df['checkpoint'] == str(checkpoint))]
    else:
      samples = df.loc[(df['checkpoint'] == str(checkpoint)) & (df['rank'] == str(rank))]
    paths = [getpath(row) for (index, row) in samples.iterrows()]
    print(f"checkpoint {checkpoint} paths size = {len(paths)}")
    images = torch.cat([torch.tensor(np.array(Image.open(path).convert("RGB"))).unsqueeze(0).permute(0, 3, 1, 2) for path in paths])
    # images.to('cuda')
    kid = KernelInceptionDistance(subset_size=24)
    gc.collect()
    kid.update(real_images, real=True)
    gc.collect()
    kid.update(images, real=False)
    gc.collect()
    computed = kid.compute()
    print(f"checkpoint={checkpoint}, rank={rank}, kid={computed}")
    results[f'r{rank}'].append(computed[0])
    if rank == 4:
      results[f'checkpoint'].append(checkpoint)
    if checkpoint == 0:
      results[f'r64'].append(computed[0])

  # df.loc[index, 'clip'] = calculate_clip_score(image, [f'{row["side"]} of fair-skinned {row["gender"]} {row["hand"]} hand'])
pd.DataFrame(results)

100%|██████████| 128/128 [00:20<00:00,  6.20it/s]


checkpoint 0 paths size = 48


/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Metric `Kernel Inception Distance` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)  # noqa: B028


checkpoint=0, rank=4, kid=(tensor(0.0911), tensor(0.0078))
checkpoint 5000 paths size = 48
checkpoint=5000, rank=4, kid=(tensor(0.0970), tensor(0.0121))
checkpoint 5000 paths size = 48
checkpoint=5000, rank=64, kid=(tensor(0.0845), tensor(0.0093))
checkpoint 10000 paths size = 48
checkpoint=10000, rank=4, kid=(tensor(0.1290), tensor(0.0156))
checkpoint 10000 paths size = 48
checkpoint=10000, rank=64, kid=(tensor(0.0834), tensor(0.0083))
checkpoint 15000 paths size = 48
checkpoint=15000, rank=4, kid=(tensor(0.1343), tensor(0.0158))
checkpoint 15000 paths size = 48
checkpoint=15000, rank=64, kid=(tensor(0.0881), tensor(0.0100))
checkpoint 20000 paths size = 48
checkpoint=20000, rank=4, kid=(tensor(0.1534), tensor(0.0180))
checkpoint 20000 paths size = 48
checkpoint=20000, rank=64, kid=(tensor(0.0895), tensor(0.0088))
checkpoint 25000 paths size = 48
checkpoint=25000, rank=4, kid=(tensor(0.1302), tensor(0.0156))
checkpoint 25000 paths size = 48
checkpoint=25000, rank=64, kid=(tensor(0.089

,checkpoint,r4,r64
0,0,tensor(0.0911),tensor(0.0911)
1,5000,tensor(0.0970),tensor(0.0845)
2,10000,tensor(0.1290),tensor(0.0834)
3,15000,tensor(0.1343),tensor(0.0881)
4,20000,tensor(0.1534),tensor(0.0895)
5,25000,tensor(0.1302),tensor(0.0898)
6,30000,tensor(0.1345),tensor(0.0987)


In [14]:
import torch_fidelity
from tqdm import tqdm
from PIL import Image
import numpy as np
import torchvision
import torch
import os
from tqdm import tqdm
import random
import gc
from torch_fidelity.fidelity import main
from functools import partial

def calculate_clip_score(image, prompts):
    # images_int = (images * 255).astype("uint8")
    # clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    clip_score = clip_score_fn(torchvision.transforms.functional.pil_to_tensor(image).unsqueeze(0), prompts).detach()
    return float(clip_score)

def getpath(row):
  if row['rank'] != 'None':
    return f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt{row["checkpoint"]}_steps30_{row["image"]}.png'
  else:
    return f'/content/drive/MyDrive/LoRA/inference/hands and palms r{row["rank"]} lora/{row["side"]}_of_fair-skinned_{row["gender"]}_{row["hand"]}_hand_chkpt5000_steps30_{row["image"]}.png'


from torchmetrics.image.kid import KernelInceptionDistance

dataset_path = '/content/drive/MyDrive/cs236 project/hands and palms/Hands/Hands'
real_paths = sorted([os.path.join(dataset_path, x) for x in os.listdir(dataset_path) if not x.endswith('.csv')])
random.seed(1234)
_ = torch.manual_seed(1234)
random.shuffle(real_paths)
real_paths = real_paths[:128]
real_images = []
for path in tqdm(real_paths):
  real_images.append(torch.tensor(np.array(Image.open(path).convert("RGB"))).unsqueeze(0).permute(0, 3, 1, 2))
gc.collect()
real_images = torch.cat(real_images)
# real_images.to('cuda')

results = {"checkpoint":[], "r4":[], "r64":[]}
for checkpoint in [0, 5000, 10000, 15000, 20000, 25000, 30000]:
  for rank in [4, 64]:
    if checkpoint ==0 and rank == 64:
      continue
    if checkpoint ==0:
      samples = df.loc[(df['checkpoint'] == str(checkpoint))]
    else:
      samples = df.loc[(df['checkpoint'] == str(checkpoint)) & (df['rank'] == str(rank))]
    paths = [getpath(row) for (index, row) in samples.iterrows()]
    print(f"checkpoint {checkpoint} paths size = {len(paths)}")
    images = torch.cat([torch.tensor(np.array(Image.open(path).convert("RGB"))).unsqueeze(0).permute(0, 3, 1, 2) for path in paths])
    # images.to('cuda')
    kid = KernelInceptionDistance(subset_size=24)
    gc.collect()
    kid.update(real_images, real=True)
    gc.collect()
    kid.update(images, real=False)
    gc.collect()
    computed = kid.compute()
    print(f"checkpoint={checkpoint}, rank={rank}, kid={computed}")
    results[f'r{rank}'].append(computed[0].item())
    if rank == 4:
      results[f'checkpoint'].append(checkpoint)
    if checkpoint == 0:
      results[f'r64'].append(computed[0].item())

  # df.loc[index, 'clip'] = calculate_clip_score(image, [f'{row["side"]} of fair-skinned {row["gender"]} {row["hand"]} hand'])
pd.DataFrame(results)

100%|██████████| 128/128 [00:02<00:00, 61.23it/s]


checkpoint 0 paths size = 48
checkpoint=0, rank=4, kid=(tensor(0.0966), tensor(0.0086))
checkpoint 5000 paths size = 48
checkpoint=5000, rank=4, kid=(tensor(0.1137), tensor(0.0173))
checkpoint 5000 paths size = 48
checkpoint=5000, rank=64, kid=(tensor(0.0962), tensor(0.0121))
checkpoint 10000 paths size = 48
checkpoint=10000, rank=4, kid=(tensor(0.1492), tensor(0.0167))
checkpoint 10000 paths size = 48
checkpoint=10000, rank=64, kid=(tensor(0.0923), tensor(0.0108))
checkpoint 15000 paths size = 48
checkpoint=15000, rank=4, kid=(tensor(0.1533), tensor(0.0178))
checkpoint 15000 paths size = 48
checkpoint=15000, rank=64, kid=(tensor(0.1003), tensor(0.0124))
checkpoint 20000 paths size = 48
checkpoint=20000, rank=4, kid=(tensor(0.1719), tensor(0.0216))
checkpoint 20000 paths size = 48
checkpoint=20000, rank=64, kid=(tensor(0.1013), tensor(0.0115))
checkpoint 25000 paths size = 48
checkpoint=25000, rank=4, kid=(tensor(0.1492), tensor(0.0192))
checkpoint 25000 paths size = 48
checkpoint=2500

,checkpoint,r4,r64
0,0,0.096569,0.096569
1,5000,0.113662,0.096158
2,10000,0.149193,0.092265
3,15000,0.153257,0.100294
4,20000,0.171914,0.101303
5,25000,0.149186,0.097251
6,30000,0.153657,0.108857


In [ ]:
df

,rank,checkpoint,side,gender,hand,image,classifier
0,64,30000,front,male,right,5,0.058778
1,64,30000,front,male,right,4,0.832572
2,64,30000,front,male,right,3,0.791907
3,64,30000,front,male,right,2,0.851860
4,64,30000,front,male,right,1,0.725522
...,...,...,...,...,...,...,...
571,4,5000,back,female,left,4,0.528272
572,4,5000,back,female,left,3,0.847858
573,4,5000,back,female,left,2,0.083410
574,4,5000,back,female,left,1,0.878268


In [ ]:
gc.open('hands and palms loras').worksheet('scale9').update('L2:L577', df[['classifier']].values.tolist())

<ipython-input-13-6cdcbb416e9e>:1: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  gc.open('hands and palms loras').worksheet('scale9').update('L2:L577', df[['classifier']].values.tolist())


{'spreadsheetId': '1GHwH5am7o7RKga8FPruSKLgMTXIWG_wEMKk__L0J72A',
 'updatedRange': 'scale9!L2:L577',
 'updatedRows': 576,
 'updatedColumns': 1,
 'updatedCells': 576}

In [ ]:
df

,rank,checkpoint,side,gender,hand,image,bad,ok,good,mediapipe_highconf,weighted human,mediapipe
0,64,30000,front,male,right,5,1,None,None,0.000000,0.0,0.792618
1,64,30000,front,male,right,4,1,None,None,0.000000,0.0,0.700885
2,64,30000,front,male,right,3,1,None,None,0.000000,0.0,0.972215
3,64,30000,front,male,right,2,,1,None,0.000000,0.5,0.584440
4,64,30000,front,male,right,1,1,None,None,0.000000,0.0,0.610859
...,...,...,...,...,...,...,...,...,...,...,...,...
619,None,0,back,female,left,4,1,None,None,0.994964,0.0,0.994930
620,None,0,back,female,left,3,,1,None,0.998482,0.5,0.998439
621,None,0,back,female,left,2,1,None,None,0.985213,0.0,0.980931
622,None,0,back,female,left,1,1,None,None,0.965399,0.0,0.974257
